In [1]:
import numpy as np
import pandas as pd

In [2]:
Books= pd.read_csv('Books.csv')
Users= pd.read_csv('Users.csv')
Ratings= pd.read_csv('Ratings.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_7564\1856948644.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  Books= pd.read_csv('Books.csv')


In [3]:
Books.head()
Users.head()
Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
print(Books.shape)
print(Users.shape)
print(Ratings.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [5]:
Books.isnull().sum()
Users.isnull().sum()
Ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

POPULARITY BASED RECOMMENDER SYSTEM on basis of books having min 250 votes and then highest avg of them... 

In [6]:
ratings_withName=Ratings.merge(Books, on="ISBN")
ratings_withName.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


In [7]:
neumeric_RatingDF=ratings_withName.groupby("Book-Title").count()['Book-Rating'].reset_index()
neumeric_RatingDF.rename(columns={'Book-Rating':'No of Rating'},inplace="true")
neumeric_RatingDF

,Book-Title,No of Rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [8]:
# ratings_withName['Book-Rating'] = pd.to_numeric(ratings_withName['Book-Rating'], errors='coerce')

In [9]:
avg_RatingDF = (
    ratings_withName.groupby("Book-Title")["Book-Rating"]
    .mean()
    .reset_index()
    .rename(columns={"Book-Rating": "Avg_Rating"})
)

avg_RatingDF.head()


,Book-Title,Avg_Rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [10]:
popular_DF=neumeric_RatingDF.merge(avg_RatingDF, on='Book-Title')
popular_DF.head()
popular_DF=popular_DF[popular_DF['No of Rating']>=250].sort_values('Avg_Rating',ascending=False).head(50)
popular_DF=popular_DF.merge(Books, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author',"Year-Of-Publication",'Image-URL-M','No of Rating','Avg_Rating']]
popular_DF.head()

,Book-Title,Book-Author,Year-Of-Publication,Image-URL-M,No of Rating,Avg_Rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,http://images.amazon.com/images/P/0439136350.0...,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,http://images.amazon.com/images/P/0439139597.0...,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,http://images.amazon.com/images/P/0590353403.0...,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,http://images.amazon.com/images/P/043935806X.0...,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,http://images.amazon.com/images/P/0439064872.0...,556,5.183453


Collaborative Filtering Based Recommender System- to filter out users and the books they rated while keepng check then only those users who have rated >200 and only those books that have been rated more than by 50 users


In [11]:
x=ratings_withName.groupby('User-ID').count()['Book-Rating']>200
FamousUsers=x[x].index  #boolean expression it now gives only true values to x


In [12]:
filtered_ratings=ratings_withName[ratings_withName['User-ID'].isin(FamousUsers)]

In [13]:
y=filtered_ratings.groupby('Book-Title').count()['Book-Rating']>50
famousBooks=y[y].index

In [14]:
final_ratings=filtered_ratings[filtered_ratings['Book-Title'].isin(famousBooks)]

In [15]:
Our_Data_Table=final_ratings.pivot_table(index='Book-Title',columns='User-ID',values="Book-Rating")
Our_Data_Table.fillna(0,inplace=True)

In [16]:
Our_Data_Table.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
Similarity_score=cosine_similarity(Our_Data_Table) 
# this is the cosine similarity of each book every other book....
# for ex= any one array is od cosine similarity of harry with every book in table

In [19]:
Similarity_score.shape

(679, 679)

In [20]:
def recommend(Book_Name):
    #fetch index
    index=np.where(Our_Data_Table.index==Book_Name)[0][0]
    similar_items=sorted(list(enumerate(Similarity_score[index])), key=lambda x: x[1], reverse=True)[1:11]
    
    for i in similar_items:
        print(Our_Data_Table.index[i[0]])
    

In [21]:
recommend('The Notebook')


A Walk to Remember
The Rescue
One Door Away from Heaven
Toxin
The Five People You Meet in Heaven
Nights in Rodanthe
Suzanne's Diary for Nicholas
Everything's Eventual : 14 Dark Tales
The Guardian
Fine Things
